In [1]:
import os
import string
import pandas as pd
import nltk
from nltk.util import bigrams, ngrams
from collections import Counter
from nltk.corpus import stopwords
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2023-11-18 16:46:54.826970: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 16:46:54.869940: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-18 16:46:54.870008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-18 16:46:54.871230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-18 16:46:54.877039: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 16:46:54.877423: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
datasets_folder = "../datasets"
file_name = "tupi_binary.csv"

In [4]:
def read_csv_file(file_name):
    file_path = os.path.join(datasets_folder, file_name)
    if os.path.isfile(file_path):
        return pd.read_csv(file_path)
    return None

In [5]:
df = read_csv_file(file_name)

In [6]:
def preprocess_text(text):
    stop_words = set(stopwords.words('portuguese'))  # Use the appropriate language for your text
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words and not any(char in word for char in ['https', '@'])]
    return filtered_tokens

In [7]:
if df is not None:
    df['text'] = df['text'].str.lower()
    df['preprocessed_text'] = df['text'].apply(preprocess_text)

In [8]:
df.head()

,source,id,text,researcher,year,aggressive,hate,preprocessed_text
0,twitter,1.65848623693028e+18,@user @user @user quanto vc pagava na época da...,oliveira et al,2023,1,1,"[user, user, user, quanto, vc, pagava, época, ..."
1,twitter,1.65848623777333e+18,@user os árabes já vão lhes chutar do país ??,oliveira et al,2023,1,1,"[user, árabes, vão, chutar, país, ?, ?]"
2,twitter,1.65848960585394e+18,@user @user @user @user @user tem que desenhar...,oliveira et al,2023,1,1,"[user, user, user, user, user, desenhar, expli..."
3,twitter,1.65849012716374e+18,@user @user chola mais gado. e se não quiser p...,oliveira et al,2023,1,1,"[user, user, chola, gado, ., quiser, pagar, ba..."
4,twitter,1.65849018793945e+18,michele micheque nao tinha cartao do bolsonaro...,oliveira et al,2023,1,1,"[michele, micheque, nao, cartao, bolsonaro, pq..."


In [9]:
exclude_terms = {'https', '@', 'user', 'link', '#', '??', '!!', '_:', '.:', '!:', '? ?', '! !', '_ :', '! :', '? :','mundo'}
exclude_punctuation = set(string.punctuation)

In [10]:
# Function to preprocess a list of strings
def preprocess_text(word_list):
    # Remove exclusion terms
    word_list = [word for word in word_list if word not in exclude_terms]

    # Remove punctuation
    word_list = [''.join(char for char in word if char not in exclude_punctuation) for word in word_list]

    return word_list

# Apply preprocessing to the 'preprocessed_text' column
df['preprocessed_text'] = df['preprocessed_text'].apply(preprocess_text)

In [11]:
df.head()

,source,id,text,researcher,year,aggressive,hate,preprocessed_text
0,twitter,1.65848623693028e+18,@user @user @user quanto vc pagava na época da...,oliveira et al,2023,1,1,"[quanto, vc, pagava, época, , vc, diz, , , can..."
1,twitter,1.65848623777333e+18,@user os árabes já vão lhes chutar do país ??,oliveira et al,2023,1,1,"[árabes, vão, chutar, país, , ]"
2,twitter,1.65848960585394e+18,@user @user @user @user @user tem que desenhar...,oliveira et al,2023,1,1,"[desenhar, explicar, desenho, pro, retardado, ..."
3,twitter,1.65849012716374e+18,@user @user chola mais gado. e se não quiser p...,oliveira et al,2023,1,1,"[chola, gado, , quiser, pagar, barato, gasolin..."
4,twitter,1.65849018793945e+18,michele micheque nao tinha cartao do bolsonaro...,oliveira et al,2023,1,1,"[michele, micheque, nao, cartao, bolsonaro, pq..."


In [15]:
# Assuming you have a DataFrame with a 'text' column and a 'label' column
texts = df['preprocessed_text'].tolist()
labels = df['hate'].tolist()

# Tokenize and preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(seq) for seq in sequences)

X = pad_sequences(sequences, maxlen=max_length, padding='post')
y = np.array(labels)  # Assuming 'labels' is a list or array of integers

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Assuming y_train and y_test are lists or arrays of 0s and 1s
# If not, you can use to_categorical to one-hot encode them
from tensorflow.keras.utils import to_categorical
y_train_one_hot = y_train  # Assuming y_train is already a list or array of 0s and 1s
y_test_one_hot = y_test  # Assuming y_test is already a list or array of 0s and 1s

In [17]:
# Build the LSTM model
embedding_dim = 50
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))  # 1 neuron for binary classification with sigmoid activation

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
model.fit(X_train, y_train_one_hot, epochs=epochs, batch_size=batch_size, validation_split=0.1)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)
print(f'Test Accuracy: {test_accuracy}')

Epoch 1/10
983/983 [==============================] - 215s 216ms/step - loss: 0.3697 - accuracy: 0.8807 - val_loss: 0.3641 - val_accuracy: 0.8815
Epoch 2/10
983/983 [==============================] - 223s 227ms/step - loss: 0.3662 - accuracy: 0.8808 - val_loss: 0.3640 - val_accuracy: 0.8815
Epoch 3/10
983/983 [==============================] - 222s 226ms/step - loss: 0.3659 - accuracy: 0.8808 - val_loss: 0.3639 - val_accuracy: 0.8815
Epoch 4/10
983/983 [==============================] - 221s 225ms/step - loss: 0.3659 - accuracy: 0.8808 - val_loss: 0.3647 - val_accuracy: 0.8815
Epoch 5/10
983/983 [==============================] - 205s 208ms/step - loss: 0.3660 - accuracy: 0.8808 - val_loss: 0.3639 - val_accuracy: 0.8815
Epoch 6/10
983/983 [==============================] - 178s 181ms/step - loss: 0.3658 - accuracy: 0.8808 - val_loss: 0.3649 - val_accuracy: 0.8815
Epoch 7/10
983/983 [==============================] - 182s 185ms/step - loss: 0.3659 - accuracy: 0.8808 - val_loss: 0.3673 -